In [40]:
from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False, show_parent=False)

enable_replay(False, False)
show_object = replay

cv = open_viewer("3dViewer", anchor="right", height=1024)


Enabling jupyter_cadquery replay


In [41]:
import svgpathtools
from svgpathtools import svg2paths
import cadquery as cq
import numpy as np
from math import sin, cos, sqrt, pi, acos, fmod, degrees

######################################################################
#  A proof of concept adding a svg path into a cadQuery Workspace
#  object.
#
#  This file is in the public domain.
#
#  Dov Grobgeld <dov.grobgeld@gmail.com>
#  2024-03-10 Sun
######################################################################


def tpl(cplx):
  '''Convert a complex number to a tuple'''
  return (cplx.real,cplx.imag)

def angle_between(u,v):
  '''Find the angle between the vectors u an v'''
  ux,uy = u
  vx,vy = v
  sign = 1 if ux*vy-uy*vx > 0 else -1
  arg = (ux*vx+uy*vy)/(sqrt(ux*ux+uy*uy)*sqrt(vx*vx+vy*vy))
  return sign*acos(arg)

# Implementation of https://www.w3.org/TR/SVG/implnote.html#ArcConversionCenterToEndpoint
def arc_endpoint_to_center(
  start,
  end,
  flag_a,
  flag_s,
  radius,
  phi):
  '''Convert a endpoint elliptical arc description to a center description'''
  rx,ry = radius.real,radius.imag
  x1,y1 = start.real,start.imag
  x2,y2 = end.real,end.imag
  
  cosphi = cos(phi)
  sinphi = sin(phi)
  rx2 = rx*rx
  ry2 = ry*ry

  # Step 1. Compute x1p,y1p
  x1p,y1p = (np.array([[cosphi,sinphi],
                       [-sinphi,cosphi]])
             @ np.array([x1-x2, y1-y2])*0.5).flatten()
  x1p2 = x1p*x1p
  y1p2 = y1p*y1p

  # Step 2: Compute (cx', cy')
  cxyp = sqrt((rx2*ry2 - rx2*y1p2 - ry2*x1p2)
              / (rx2*y1p2 + ry2*x1p2)) * np.array([rx*y1p/ry,-ry*x1p/rx])

  if flag_a == flag_s:
    cxyp = -cxyp

  cxp,cyp = cxyp.flatten()

  # Step 3: compute (cx,cy) from (cx',cy')
  cx,cy = (cosphi*cxp - sinphi * cyp + 0.5*(x1+x2),
           sinphi*cxp + cosphi * cyp + 0.5*(y1+y2))

  # Step 4: compute theta1 and deltatheta
  theta1 = angle_between((1,0), ((x1p-cxp)/rx, (y1p-cyp)/ry))
  delta_theta = fmod(angle_between(((x1p-cxp)/rx,(y1p-cyp)/ry),
                                   ((-x1p-cxp)/rx, (-y1p-cyp)/ry)),2*pi)

  # Choose the right edge according to the flags
  if not flag_s and delta_theta > 0:
    delta_theta -= 2*pi
  elif flag_s and delta_theta < 0:
    delta_theta += 2*pi
    
  return (cx,cy), theta1, delta_theta

def compare_complex(num1, num2, tolerance):
  """
  Compares two complex numbers using a given tolerance for both the real and imaginary parts.

  :param num1: The first complex number to be compared.
  :param num2: The second complex number to be compared.
  :param tolerance: The tolerance to use when comparing the real and imaginary parts of the numbers.

  :return: True if the real and imaginary parts of the two complex numbers are within the given tolerance, False otherwise.
  """
  # Check that the input is valid
  if not (isinstance(num1, complex) and isinstance(num2, complex)):
    raise ValueError("Both inputs must be complex numbers")

  # Check that the tolerance is valid
  if not (isinstance(tolerance, float) or isinstance(tolerance, int)):
    raise ValueError("Tolerance must be a number")

  # Check that the real and imaginary parts of the two complex numbers are within the given tolerance
  if abs(num1.real - num2.real) < tolerance and abs(num1.imag - num2.imag) < tolerance:
     return True

  return False

def addSvgPath(self, path):
  '''
  Add the svg path object to the current workspace
  The p in path below is each movement in the path, where the movement
  can be a line, cubic/quad curve, or arc.
  
  The first p must be either a point or a move to that point.
  
  All p's are translated using bezier, ellipseArc commands and added
  to res (I assume res = result).
  '''

  print('Start Path')

  res = self
  path_start = None
  arc_id = 0
  for p in path:
    #print('path element to add: ',p)
    if path_start is None:
        path_start = p.start
    res = res.moveTo(*tpl(p.start))

    # Support the four svgpathtools different objects
    if isinstance(p, svgpathtools.CubicBezier):
      coords = (tpl(p.start), tpl(p.control1), tpl(p.control2), tpl(p.end))
      res = res.bezier(coords)
    elif isinstance(p, svgpathtools.QuadraticBezier):
      coords = (tpl(p.start), tpl(p.control), tpl(p.end))
      res = res.bezier(coords)
      pass
    elif isinstance(p, svgpathtools.Arc):
      arc_id += 1
      center,theta1,delta_theta = arc_endpoint_to_center(
        p.start,
        p.end,
        p.large_arc,
        p.sweep,
        p.radius,
        p.rotation)

      res = res.ellipseArc(
        x_radius = p.radius.real,
        y_radius = p.radius.imag,
        rotation_angle=degrees(p.rotation),
        angle1= degrees(theta1),
        angle2=degrees(theta1+delta_theta)
        )
    elif isinstance(p, svgpathtools.Line):   
      #Check to see if start and end points are the same - 0.001
      samePoint = compare_complex(p.start, p.end, 0.001)
      if samePoint:
         #res = res.lineTo(p.end.real, p.end.imag)
         pass
         #print('Start and End are the same - skipping')
      else:
        #print('Adding line')
        res = res.lineTo(p.end.real, p.end.imag)
    
    else:
        print('Some other path type')
      
    if path_start == p.end:
      path_start = None
      res = res.close()

  return res

cq.Workplane.addSvgPath = addSvgPath

#Load and print number of paths

#Working
#paths, attributes = svg2paths('noun-gutenberg-press-86670-cleaned.svg')
#paths, attributes = svg2paths('noun-houseplants-886781-edited.svg')
#paths, attributes = svg2paths('noun-ornaments-714185-cleaned.svg')
#paths, attributes = svg2paths('noun-birthday-5954448.svg')
paths, attributes = svg2paths('noun-houseplants-6613684.svg')

#Not working
#paths, attributes = svg2paths('noun-shape-2369275.svg')
#paths, attributes = svg2paths('nomnoml-SVGOMG.svg')
#paths, attributes = svg2paths('noun-complex-decoration-3061208.svg')
#paths, attributes = svg2paths('noun-complex-flower-778213.svg')

#To test/verify
#paths, attributes = svg2paths('noun-door-open-5938229.svg') #Can't tell if some cuts are going the wrong way


print('Paths Found: %s' % format(len(paths)))

#Create work plane, draw SVG, and extrude
guten = cq.Workplane('XY').tag('workFace')
for idx, path in enumerate(paths):
  print(idx)
  #Skip path if needed. If not, set to some high number.
  if idx < 50:
    svgPath = (
        guten
        .workplaneFromTagged('workFace')
        .addSvgPath(path)
        .extrude(10, clean=True)
    )
    guten = guten.union(svgPath, glue = True)
    #show_object(guten)

print('Done Building SVG')

bbox = guten.val().BoundingBox()

Paths Found: 1
0
Start Path
Done Building SVG


In [42]:
print(bbox.center.x)
print(bbox.xlen)
print(bbox.ylen)

47.898557677373134
50.84826656024117
89.13139732669647


In [43]:
# https://stackoverflow.com/questions/77845206/cadquery-unexpected-valueerror-null-topods-shape-object-during-cut-operation
guten = (
    guten.workplaneFromTagged('workFace').workplane(offset = 0.0001)
    .center(bbox.center.x, bbox.center.y)
    .rect(bbox.xlen*1.25, bbox.ylen*1.25)
    .extrude(-100, combine='a', clean=True)    
)

In [44]:
show_object(guten)

Use the multi select box below to select one or more steps you want to examine
